<a id='top'></a>

# CSCI 3202, Spring 2020
# Assignment 3
# Due:  Wednesday 4 March 2020 by 11:59 PM

<br>

### Your name: Alex Book

<br>




**Some things that might be useful**


In [1]:
import pandas as pd
import numpy as np
import copy as cp
from scipy import stats
from math import floor
import matplotlib.pyplot as plt
import csv
from time import time

<br>

---

## Problem 1 (35 points):  Playing "intelligent" Tic-Tac-Toe

<img src="https://www.cookieshq.co.uk/images/2016/06/01/tic-tac-toe.png" width="150"/>

<a id='p2a'></a>

### (1a)   Defining the Tic-Tac-Toe class structure

Fill in this class structure for Tic-Tac-Toe using what we did during class as a guide.
* `moves` is a list of tuples to represent which moves are available. Recall that we are using matrix notation for this, where the upper-left corner of the board, for example, is represented at (1,1).
* `result(self, move, state)` returns a ***hypothetical*** resulting `State` object if `move` is made when the game is in the current `state`
* `compute_utility(self, move, state)` calculates the utility of `state` that would result if `move` is made when the game is in the current `state`. This is where you want to check to see if anyone has gotten `nwin` in a row
* `game_over(self, state)` - this wasn't a method, but it should be - it's a piece of code we need to execute repeatedly and giving it a name makes clear what task the piece of code performs. Returns `True` if the game in the given `state` has reached a terminal state, and `False` otherwise.
* `utility(self, state, player)` also wasn't a method earlier, but also should be.  Returns the utility of the current state if the player is X and $-1 \cdot$ utility if the player is O.
* `display(self)` is a method to display the current game `state`, You get it for free! because this would be super frustrating without it.
* `play_game(self, player1, player2)` returns an integer that is the utility of the outcome of the game (+1 if X wins, 0 if draw, -1 if O wins). `player1` and `player2` are functional arguments that we will deal with in parts **2b** and **2d**.

Some notes:
* Assume X always goes first.
* Do **not** hard-code for 3x3 boards.
* You may add attributes and methods to these classes as needed for this problem.

In [2]:
# Fill in as needed.
class State:
    def __init__(self, moves):
        self.to_move = 'X'
        self.utility = 0
        self.board = {}
        self.moves = cp.copy(moves)

        
class TicTacToe:
    
    def __init__(self, nrow=3, ncol=3, nwin=3, nexp=0):
        self.nrow = nrow
        self.ncol = ncol
        self.nwin = nwin
        moves = [(row, col) for row in range(1, nrow + 1) for col in range(1, ncol + 1)]
        self.state = State(moves)
        

    def result(self, move, state):
        '''
        What is the hypothetical result of move `move` in state `state` ?
        move  = (row, col) tuple where player will put their mark (X or O)
        state = a `State` object, to represent whose turn it is and form
                the basis for generating a **hypothetical** updated state
                that will result from making the given `move`
        '''

        # Don't do anything if the move isn't a legal one
        if move not in state.moves:
            return state
        # Return a copy of the updated state:
        #   compute utility, update the board, remove the move, update whose turn
        new_state = cp.deepcopy(state)
        new_state.utility = self.compute_utility(move, state)
        new_state.board[move] = state.to_move
        new_state.moves.remove(move)
        new_state.to_move = ('O' if state.to_move == 'X' else 'X')
        return new_state
    
    def compute_utility(self, move, state):
        '''
        What is the utility of making move `move` in state `state`?
        If 'X' wins with this move, return 1;
        if 'O' wins return -1;
        else return 0.
        '''        

        row, col = move
        player = state.to_move
        
        # create a hypothetical copy of the board, with 'move' executed
        board = cp.deepcopy(state.board)
        board[move] = player

        # what are all the ways 'player' could with with 'move'?
        
        # check for row-wise win
        in_a_row = 0
        for c in range(1,self.ncol+1):
            in_a_row += board.get((row,c))==player

        # check for column-wise win
        in_a_col = 0
        for r in range(1,self.nrow+1):
            in_a_col += board.get((r,col))==player

        # check for NW->SE diagonal win
        in_a_diag1 = 0
        for r in range(row,0,-1):
            in_a_diag1 += board.get((r,col-(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag1 += board.get((r,col-(row-r)))==player

        # check for SW->NE diagonal win
        in_a_diag2 = 0
        for r in range(row,0,-1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        for r in range(row+1,self.nrow+1):
            in_a_diag2 += board.get((r,col+(row-r)))==player
        
        if self.nwin in [in_a_row, in_a_col, in_a_diag1, in_a_diag2]:
            return 1 if player=='X' else -1
        else:
            return 0
        

    def game_over(self, state):
        '''game is over if someone has won (utility!=0) or there
        are no more moves left'''

        return state.utility!=0 or len(state.moves)==0
        
           

    
    def utility(self, state, player):
        '''Return the value to player; 1 for win, -1 for loss, 0 otherwise.'''

        return state.utility if player=='X' else -state.utility
        
                
        
        
    def display(self):
        board = self.state.board
        for row in range(1, self.nrow + 1):
            for col in range(1, self.ncol + 1):
                print(board.get((row, col), '.'), end=' ')
            print()
        
    def play_game(self, player1, player2):
        '''Play a game of tic-tac-toe!'''

        turn_limit = self.nrow*self.ncol  # limit in case of buggy code
        turn = 0
        while turn<=turn_limit:
            for player in [player1, player2]:
                turn += 1
                move = player(self)
                self.state = self.result(move, self.state)
                if self.game_over(self.state):
#                     self.display()
                    return self.state.utility           



<br>


### (1b) Define a random player

Define a function `random_player` that takes a single argument of the `TicTacToe` class and returns a random move out of the available legal moves in the `state` of the `TicTacToe` game.

In your code for the `play_game` method above, make sure that `random_player` could be a viable input for the `player1` and/or `player2` arguments.

In [3]:
def random_player(game):
    '''A player that chooses a legal move at random out of all
    available legal moves in Tic-Tac-Toe state argument'''

    
    possible_actions = game.state.moves
    return possible_actions[np.random.randint(low=0, high=len(possible_actions))]


<br>


### (1c) What about playing randomly on different-sized boards?

What does the long-term win percentage appear to be for the first player in a 4x4 Tic-Tac-Toe tournament, where 4 marks must be connected for a win?  Support your answer using a simulation and printed output, similar to **2b**.

**Also:** The win percentage should have changed substantially. Did the decrease in wins turn into more losses for the first player or more draws? Write a few sentences explaining the behavior you observed.  *Hint: think about how the size of the state space has changed.*

In [4]:
# Your code here.
runs = 10000

three_results = []
print("3x3")
for i in range(runs):
    ex = TicTacToe(3,3,3)
    three_results.append(ex.play_game(random_player, random_player))
print("Win pct:", three_results.count(1)/runs)
print("Draw pct:", three_results.count(0)/runs)
print("Loss pct:", three_results.count(-1)/runs)

print()
four_results = []
print("4x4")
for i in range(runs):
    ex2 = TicTacToe(4,4,4)
    four_results.append(ex2.play_game(random_player, random_player))
print("Win pct:", four_results.count(1)/runs)
print("Draw pct:", four_results.count(0)/runs)
print("Loss pct:", four_results.count(-1)/runs)

3x3
Win pct: 0.5879
Draw pct: 0.1269
Loss pct: 0.2852

4x4
Win pct: 0.3096
Draw pct: 0.4206
Loss pct: 0.2698


The larger the board size, the lesser the chance is of a player randomly picking the satisfactory number of spaces in a row to win. Therefore the long-term win percentage decreases, mainly in lieu of the long-term draw percentage increasing.

<br>


### (1d) Define an alpha-beta player

Alright. Let's finally get serious about our Tic-Tac-Toe game.  No more fooling around!

Craft a function called `alphabeta_player` that takes a single argument of a `TicTacToe` class object and returns the minimax move in the `state` of the `TicTacToe` game. As the name implies, this player should be implementing alpha-beta pruning as described in the textbook and lecture.

Note that your alpha-beta search for the minimax move should include function definitions for `max_value` and `min_value` (see the aggressively realistic pseudocode from the lecture slides).

In your code for the `play_game` method above, make sure that `alphabeta_player` could be a viable input for the `player1` and/or `player2` arguments.

In [5]:
# Your code here.
def alphabeta_player(game, expanded = False):
    return alphabeta_search(game, expanded)

def alphabeta_search(game, expanded):
    '''search game approach to find best action, using alpha-beta pruning:
    alpha = best (highest) move found so far for Max
    beta  = best (lowest) move found so far for Min'''

    expanded_count = [0]
    
    player = 1 if game.state.to_move == 'X' else -1 # X player trying to positively maximize, O player trying to negatively maximize

    # Functions used by alphabeta
    def max_value(state, alpha, beta):
        expanded_count[0] += 1
        if game.game_over(state):
            return (state.utility*player, None)
        # start of alpha-beta process
        value = -float('inf')
        best = None
        for move in state.moves:
            newValue, _ = min_value(game.result(move, state), alpha, beta)
            if newValue > value:
                best = move
                value = newValue
            if value >= beta:
                return (value, best)
            alpha = max(value, alpha)
        return (value, best)

    def min_value(state, alpha, beta):
        expanded_count[0] += 1
        if game.game_over(state):
            return (state.utility*player, None)
        # start of alpha-beta process
        value = float('inf')
        best = None
        for move in state.moves:
            newValue, _ = max_value(game.result(move, state), alpha, beta)
            if newValue < value:
                best = move
                value = newValue
            if value <= alpha:
                return (value, best)
            beta = min(value, beta)
        return (value, best)
        
    # Body of alphabeta_cutoff_search:
    alpha = -float('inf')
    beta = float('inf')
    _, action = max_value(game.state, alpha, beta)
    if expanded is True:
        return expanded_count[0]
    return action

Verify that your alpha-beta player code is working appropriately through the following tests, using a standard 3x3 Tic-Tac-Toe board. Run **only 10 games for each test**, and track the number of wins, draws and losses.

1. An alpha-beta player who plays first should never lose to a random player who plays second.
2. A random player who plays first should never win to an alpha-beta player who plays second.
3. Two alpha-beta players should always draw.

**Nota bene:** Test your code with fewer games between the players to start, because the alpha-beta player will require substantially more compute time than the random player.  This is why I only ask for 10 games, which still might take a minute or two. 

In [6]:
# Your code here.
runs = 10

one_results = []
print("alpha-beta vs. random")
for i in range(runs):
    ex = TicTacToe(3,3,3)
    one_results.append(ex.play_game(alphabeta_player, random_player))
print("Win pct:", one_results.count(1)/runs)
print("Draw pct:", one_results.count(0)/runs)
print("Loss pct:", one_results.count(-1)/runs)

two_results = []
print("random vs. alpha-beta")
for i in range(runs):
    ex = TicTacToe(3,3,3)
    two_results.append(ex.play_game(random_player, alphabeta_player))
print("Win pct:", two_results.count(1)/runs)
print("Draw pct:", two_results.count(0)/runs)
print("Loss pct:", two_results.count(-1)/runs)

three_results = []
print("alpha-beta vs. alpha-beta")
for i in range(runs):
    ex = TicTacToe(3,3,3)
    three_results.append(ex.play_game(alphabeta_player, alphabeta_player))
print("Win pct:", three_results.count(1)/runs)
print("Draw pct:", three_results.count(0)/runs)
print("Loss pct:", three_results.count(-1)/runs)

alpha-beta vs. random
Win pct: 1.0
Draw pct: 0.0
Loss pct: 0.0
random vs. alpha-beta
Win pct: 0.0
Draw pct: 0.2
Loss pct: 0.8
alpha-beta vs. alpha-beta
Win pct: 0.0
Draw pct: 1.0
Loss pct: 0.0


<br>


### (1e) What has pruning ever done for us?

Calculate the number of number of states expanded by the minimax algorithm, with and without pruning, to determine the minimax decision from the initial 3x3 Tic-Tac-Toe board state.  This can be done in many ways, but writing out all the states by hand is **not** one of them (as you will find out!).

Write a sentence or two, commenting on the difference in number of nodes expanded by each search.

In [7]:
def minimax(game):
    '''search game approach to find best action, using alpha-beta pruning:
    alpha = best (highest) move found so far for Max
    beta  = best (lowest) move found so far for Min'''

    expanded_count = [0]
    
    player = 1 if game.state.to_move == 'X' else -1

    # Functions used by alphabeta
    def max_value(state):
        expanded_count[0] += 1
        if game.game_over(state):
            return state.utility*player
        value = -float('inf')
        for move in state.moves:
            value = max(value,min_value(game.result(move, state)))
        return value

    def min_value(state):
        expanded_count[0] += 1
        if game.game_over(state):
            return state.utility*player
        value = float('inf')
        for move in state.moves:
            value = min(value,max_value(game.result(move, state)))
        return value
        
    max_value(game.state)
    return expanded_count[0]

ex = TicTacToe(3,3,3)

print("alpha-beta")
print(alphabeta_player(ex, True))

print("minimax")
print(minimax(ex))

alpha-beta
18297
minimax
549946


Minimax has roughly 30 times the number of nodes expanded when compared to alpha-beta pruning, which makes sense, as pruning allows you to effectively "skip" nodes.

<br>

---

## Problem 2 (30 points):  Uncertainty

### (2a) 

Suppose you are deciding when to arrive at a party. There is some optimal time to arrive when the loss you feel, as measured by _awkwardness_, is minimized at 0. That is, at some particular time, it is not awkward at all to show up to the party. The awkwardness (loss) increases as you arrive too early or too late relative to this optimal time. What is a suitable loss function, $L(d, x)$, to model this situation? Include definitions for $d$ and $x$, consistent with the examples from this class. Use this loss function this weekend when you go out.

$$ L(d,x) =  \begin{cases} 
      x-d & x > d \\
      d-x & x \leq d
   \end{cases} $$
   
d represents the time of arrival at the party (starting at d=0) and x represents the current state of the party (likely how the attendees would judge your arrival).

### (2b)

Suppose we have a situation where loss is given by the function $L(d, x) = 2(d-x)^2$. Set up, simplify, and evaluate integral(s) for the expected loss, $E_x[L(d, x)]$, where your prior beliefs regarding $x$ follow the distribution $f(x)$ given below. You may assume $f(x)=0$ for values of $x$ outside of the interval $[0, 3]$.

$$f(x) =  \begin{cases} 
      1/2 & 0 \leq x <1 \\
      3/8 & 1 \leq x <2 \\
      1/8 & 2 \leq x \leq 3 
   \end{cases}$$


$E_x[L(d, x)] = \int_{0}^{3} 2(d-x)^2 dx$ \
$E_x[L(d, x)] = \int_{0}^{1} 2(d-x)^2*\frac{1}{2} dx$ + $\int_{1}^{2} 2(d-x)^2*\frac{3}{8} dx$ + $\int_{2}^{3} 2(d-x)^2*\frac{1}{8} dx$ \
$E_x[L(d, x)] = \int_{0}^{1} (d-x)^2 dx$ + $\frac{3}{4}\int_{1}^{2} (d-x)^2 dx$ + $\frac{1}{4}\int_{2}^{3} (d-x)^2 dx$ \
$E_x[L(d, x)] = \int_{0}^{1} (d^2-2dx+x^2) dx$ + $\frac{3}{4}\int_{1}^{2} (d^2-2dx+x^2) dx$ + $\frac{1}{4}\int_{2}^{3} (d^2-2dx+x^2) dx$ \
$E_x[L(d, x)] = (d^2x-dx^2+\frac{1}{3}x^3) \Big|_{0}^{1}$ + $\frac{3}{4}(d^2x-dx^2+\frac{1}{3}x^3) \Big|_{1}^{2}$ + $\frac{1}{4}(d^2x-dx^2+\frac{1}{3}x^3) \Big|_{2}^{3}$ \
$E_x[L(d, x)] = (d^2-d+\frac{1}{3})+\frac{3}{4}( (2d^2-4d+\frac{8}{3})-(d^2-d+\frac{1}{3}) )+\frac{1}{4}( (3d^2-9d+9)-(2d^2-4d+\frac{8}{3}) )$ \
$E_x[L(d, x)] = (d^2-d+\frac{1}{3})+\frac{3}{4}(d^2-3d+\frac{7}{3})+\frac{1}{4}(d^2-5d+\frac{19}{3})$ \
$E_x[L(d, x)] = (d^2-d+\frac{1}{3})+(\frac{3}{4}d^2-\frac{9}{4}d+\frac{7}{4})+(\frac{1}{4}d^2-\frac{5}{4}d+\frac{19}{12})$ \
$E_x[L(d, x)] = 2d^2-\frac{9}{2}d+\frac{11}{3}$

### (2c) 

Suppose our expected loss is represented by the function $E_x[(L(d,x)]=(2-d)^2+2$, and our prior beliefs regarding $x$ are given by the distribution $f(x)$ from part b.

- Calculate Bayes' Decision, $d_{Bayes}$.

- Calculate the Expected Value of Including Uncertainty, EVIU. Suppose that if we ignore uncertainty, our best guess for what decision to make is the median of $x$ (under our prior $f(x)$).

$d_{Bayes} = 2$ \
$d_{iu}=median(x)=1$ because the interval [0,1] contains half the volume/value of the probability distribution function \
\
$EVIU = \int_{0}^{3} (L(d_{iu},x)-L(d_{Bayes},x))f(x)dx$ \
$EVIU = \int_{0}^{3} (((2-1)^2+2) - ((2-2)^2+2))f(x)dx$ \
$EVIU = \int_{0}^{3} (3-2)f(x)dx$ \
$EVIU = 1$

<br>

## Problem 3 (35 points): Maximizing some objective function with a Genetic Algorithm

Suppose we are trying to figure out a cookie recipe, but can't quite remember how much of each ingredient we need.

So we want to maximize the following objective function corresponding to how close we are to this recipe:

* 3/4 cup granulated sugar (36 tsp)
* 3/4 cup packed brown sugar  (36 tsp)
* 1 cup butter (48 tsp)
* 1 teaspoon vanilla (1 tsp)
* 1 egg
* 2 1/4 cups flour (108 tsp)
* 1 teaspoon baking soda (1 tsp)
* 1/2 teaspoon salt (0.5 tsp)
* 1 package (12 ounces) chocolate chips (2 cups) (96 tsp)

In [8]:
# [64,32,16,8,4,2,1,.5]

# use binary to represent recipes in order to have a more "orderly" process of mutation as shown by an example in class

target = np.array([[0,1,0,0,1,0,0,0],
          [0,1,0,0,1,0,0,0],
          [0,1,1,0,0,0,0,0],
          [0,0,0,0,0,0,1,0],
          [0,0,0,0,0,0,1,0],
          [1,1,0,1,1,0,0,0],
          [0,0,0,0,0,0,1,0],
          [0,0,0,0,0,0,0,1],
          [1,1,0,0,0,0,0,0]])

# target = [36, 36, 48, 1, 1, 108, 1, 0.5, 96]

target = np.reshape(target,72) # make target recipe a 1d array for more easily checking the fitness of each recipe in a given iteration/generation
# print(target)

An example starting state for a member of our population might look like: $state = [30, 30, 40, 1, 0, 100, 0.5, 0.5, 100]$

### (3a) 

Write an objective function `def recipe_success(state)` that takes a single argument state, and returns the objective function value (fitness) of the state. The objective function should be maximized when a state reaches the target. You could for example define the fitness score of a particular state based on how far away each entry is from the target recipe.

In [9]:
# returns number of bits that are correct (maximum of 72)
def recipe_success(state):
    # Your code here.
    ret = 0
    for i in range(len(state)):
        if state[i] == target[i]:
            ret+=1
    return ret

# a = np.random.randint(0,2,72)
# print(a)
# print(recipe_success(a))
# print(recipe_success(target))

### (3b) 

Using our in class notebook "CSCI3202_GeneticAlgorithm.ipynb" as your guide, write a genetic algorithm that starts with a population of 10 randomly generated "recipes/states/members" and uses the objective function you wrote in **(3a)** to hopefully hit the target after a certain number of generations. 

Key components of your code:
- Generate the initial population randomly from numbers between 0 and 108 (half step intervals might be helpful since the recipe requires 1/2 tsp salt)
- Allow for mutations in your population with an overall probability of mutation set to p = 0.1
- Choose 2 "parents" in the generation of each "child"
- Choose a random split point at which to combine the two "parents"
- Run the algorithm for 200 iterations ("generations"). Do you hit your target?

In [10]:
# Your code here.
class problem:
    
    def __init__(self, initial_population, objective_function, mutation_probability, fitness_goal):
        '''
        initial_population = list of lists; each sub-list is a dna string for a population member
        objective_function = objective function to maximize
        mutation_probability = probability that any given child has a mutation
        fitness_goal = fitness goal to achieve (stopping criterion, once member reaches this)
        '''
        self.population = initial_population
        self.initial_population = initial_population
        self.objective_function = objective_function
        self.p_mutate = mutation_probability
        self.n_pop = len(initial_population)
        self.n_dna = len(initial_population[0])
        self.fitness_goal = fitness_goal

    def fitness(self):
        '''
        calculate each population member's probability of being selected for
        reproduction based on performance on objective function
        '''
        performance = []
        for k in range(self.n_pop):
            performance.append(self.objective_function(self.population[k]))
        return performance # returns list of the number of correct bits of each recipe in a given population/generation
        
    def reproduce(self, parent1, parent2):
        # last DNA snippet from parent1
        split = np.random.randint(low=1, high=self.n_dna)
        child = np.append(parent1[:split],parent2[split:])
        return child

    def mutate(self, child):
        # which gene to mutate?
        # will randomly flip one bit of a given individual
        gene = np.random.randint(low=0, high=self.n_dna) # picks the bit to flip
        child[gene] = 0 if child[gene]==1 else 1 # flips the bit
        return child

def genetic_algorithm(problem, n_iter):
    
    for t in range(n_iter):
        
        new_generation = []
        # select best half of population for reproduction
        performance = problem.fitness() # array of fitness levels of all members of current/parent population
        half = int(problem.n_pop/2) # number of parents that will have the possibility of reproducing
        parent_indices = np.argsort(performance)[-half:] # array of the indices of the top half of parents
        
        for k in range(problem.n_pop):
            
            indices_of_parents = np.random.choice(parent_indices, size=2, replace=False) # indices of the two parents randomly chosen from the possibilities
            parent1, parent2 = problem.population[indices_of_parents[0]], problem.population[indices_of_parents[1]] # gets the parents with the given indices
            
            # reproduce
            child = problem.reproduce(parent1, parent2)
            
            # mutate
            l_mutate = np.random.choice([True, False], p=[problem.p_mutate, 1-problem.p_mutate])
            if l_mutate:
                child = problem.mutate(child)
            
            # add to new generation
            new_generation.append(child)
        
        # set problem.population = new generation
        problem.population = new_generation
        
        # exit criterion check
        # array of fitness levels of all members of new population
        performance = [problem.objective_function(member) for member in problem.population]
        
        best_member = max(zip(performance, problem.population), key = lambda x: x[0])
        
        # checks if the best member matches the target recipe
        if best_member[0] >= problem.fitness_goal:
            return True,best_member,t # returns True, the most successful recipe, and the number of iterations taken to reach the goal

    return False,best_member,t # returns False, the most successful recipe, and the number of iterations attempted

initial_pop = []
for i in range(10): # initial population size of 10
    initial_pop.append(np.random.choice(range(2),size=72)) # 9 ingredients of 8 bits each, 72 bits total for each individual

p = problem(initial_pop, recipe_success, .1, 72) # 10% chance of mutation, maximum fitness level of 72 (if recipe matches the target)
success, best_recipe, iterations = genetic_algorithm(p, 200)

# make the recipe easily readable by changing from binary to decimal
best_recipe_reshaped = np.reshape(best_recipe[1],(9,8))
true_recipe = []
for i in best_recipe_reshaped:
    temp = int("".join(str(x) for x in i[0:7]), 2)
    if i[7] == 1:
        temp += .5
    true_recipe.append(temp)

if success == True:
    print("Reached goal state in",iterations+1,"iterations.")
    
elif success == False:
    print("Did not reach goal state after",iterations+1,"iterations.")
    
print("Best recipe:",true_recipe)

Did not reach goal state after 200 iterations.
Best recipe: [37, 37, 48, 1, 1, 108, 65, 0.5, 96]


If the randomness of my genetic algorithm "gets lucky," under 200 iterations is possible, but it's not likely (seen from my testing).

### (3c)

Report the following:
- How many generations did it take to hit the goal?
- If you change the initial population size to 100, does that change the number of generations it takes to achieve the goal recipe?
- If you change the probability of mutation to 0.2, does that affect the number of generations it takes to achieve the goal recipe?

In [11]:
# Your code here.

print("***NOTE***: The number of iterations will vary from test to test due to the randomness of the initial population, breeding, and mutation.")
print()

initial_pop = []
for i in range(10):
    initial_pop.append(np.random.choice(range(2),size=72))

p = problem(initial_pop, recipe_success, .1, 72)
success, best_recipe, iterations = genetic_algorithm(p, 1000)

best_recipe_reshaped = np.reshape(best_recipe[1],(9,8))
true_recipe = []
for i in best_recipe_reshaped:
    temp = int("".join(str(x) for x in i[0:7]), 2)
    if i[7] == 1:
        temp += .5
    true_recipe.append(temp)
    
print("It took",iterations,"iterations to hit the goal with an initial population size of 10.")
print()

########

initial_pop = []
for i in range(100):
    initial_pop.append(np.random.choice(range(2),size=72))

p = problem(initial_pop, recipe_success, .1, 72)
success, best_recipe, iterations = genetic_algorithm(p, 1000)

best_recipe_reshaped = np.reshape(best_recipe[1],(9,8))
true_recipe = []
for i in best_recipe_reshaped:
    temp = int("".join(str(x) for x in i[0:7]), 2)
    if i[7] == 1:
        temp += .5
    true_recipe.append(temp)

print("With the initial population as size 100, it now takes",iterations,"iterations to hit the goal. So increasing the size of the initial population decreases the number of iterations needed to hit the goal.")
print()

########

p = problem(initial_pop, recipe_success, .2, 72)
success, best_recipe, iterations = genetic_algorithm(p, 1000)

best_recipe_reshaped = np.reshape(best_recipe[1],(9,8))
true_recipe = []
for i in best_recipe_reshaped:
    temp = int("".join(str(x) for x in i[0:7]), 2)
    if i[7] == 1:
        temp += .5
    true_recipe.append(temp)
    
print("With a probability of mutation of .2 instead of the original .1, and using the same initial population as the test directly above, it takes",iterations,"iterations to hit the goal. So increasing the probability of mutation doesn't noticeably change the number of iterations needed to hit the goal.")

***NOTE***: The number of iterations will vary from test to test due to the randomness of the initial population, breeding, and mutation.

It took 437 iterations to hit the goal with an initial population size of 10.

With the initial population as size 100, it now takes 39 iterations to hit the goal. So increasing the size of the initial population decreases the number of iterations needed to hit the goal.

With a probability of mutation of .2 instead of the original .1, and using the same initial population as the test directly above, it takes 36 iterations to hit the goal. So increasing the probability of mutation doesn't noticeably change the number of iterations needed to hit the goal.


***NOTE***: The number of iterations will vary from test to test due to the randomness of the initial population, breeding, and mutation.